## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Souillac,MU,-20.5167,57.5167,73.78,94,75,4.90,broken clouds
1,Punta Arenas,CL,-53.1500,-70.9167,39.31,60,40,18.21,scattered clouds
2,Kokstad,ZA,-30.5472,29.4241,50.68,73,40,3.49,scattered clouds
3,Hermanus,ZA,-34.4187,19.2345,52.00,91,100,3.09,overcast clouds
4,Zeya,RU,53.7500,127.2667,41.05,85,100,3.49,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,Rikitea,PF,-23.1203,-134.9692,76.55,70,67,22.79,broken clouds
14,Eyl,SO,7.9803,49.8164,79.90,76,7,19.73,clear sky
17,Padang,ID,-0.9492,100.3543,79.99,87,93,4.41,overcast clouds
20,Kalmunai,LK,7.4167,81.8167,81.09,67,100,8.41,overcast clouds
24,Georgetown,MY,5.4112,100.3354,83.89,86,20,1.14,few clouds
25,Tall Kayf,TR,37.0833,41.0667,78.80,14,0,9.22,clear sky
27,Mahibadhoo,MV,3.7833,72.9667,83.46,75,100,22.08,overcast clouds
30,Owando,CG,-0.4819,15.8999,78.76,69,88,1.14,overcast clouds
32,Candido Mendes,BR,-1.4467,-45.7167,84.27,72,52,6.42,light rain
50,Hithadhoo,MV,-0.6000,73.0833,84.34,70,100,21.27,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   232
Country                232
Lat                    232
Lng                    232
Max Temp               232
Humidity               232
Cloudiness             232
Wind Speed             232
Current Description    232
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Rikitea,PF,76.55,broken clouds,-23.1203,-134.9692,
14,Eyl,SO,79.90,clear sky,7.9803,49.8164,
17,Padang,ID,79.99,overcast clouds,-0.9492,100.3543,
20,Kalmunai,LK,81.09,overcast clouds,7.4167,81.8167,
24,Georgetown,MY,83.89,few clouds,5.4112,100.3354,
25,Tall Kayf,TR,78.80,clear sky,37.0833,41.0667,
27,Mahibadhoo,MV,83.46,overcast clouds,3.7833,72.9667,
30,Owando,CG,78.76,overcast clouds,-0.4819,15.8999,
32,Candido Mendes,BR,84.27,light rain,-1.4467,-45.7167,
50,Hithadhoo,MV,84.34,overcast clouds,-0.6000,73.0833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
cur_desc = hotel_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))